In [1]:
import os 

os.chdir("../")
os.getcwd()

'c:\\Users\\HP\\Desktop\\flight-comptetion\\NNights-team'

In [2]:
# imports
import numpy as np
import pandas as pd

from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, 
                              GradientBoostingRegressor, AdaBoostRegressor)
from sklearn.model_selection import (GridSearchCV, 
                                     train_test_split, cross_val_score)

In [3]:
# load data
flights = pd.read_csv("data/flights_enriched_full.csv")

In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8896 entries, 0 to 8895
Data columns (total 46 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   flight_date                                       8896 non-null   object 
 1   from                                              8896 non-null   object 
 2   to                                                8896 non-null   object 
 3   avg_weeks                                         8896 non-null   float64
 4   target                                            8896 non-null   float64
 5   std_weeks                                         8896 non-null   float64
 6   is_holiday                                        8896 non-null   bool   
 7   day_of_year                                       8896 non-null   int64  
 8   distance_to_New Year's Day                        8896 non-null   int64  
 9   distance_to_Birthda

In [5]:
# select cols
useless_cols = ["ATL", "BOS", "CLT", "DEN", "DFW", 
                "DTW", "EWR", "IAH", "JFK", "LAS", "LAX", "LGA", 
                "MCO", "MIA", "MSP", "ORD", "PHL", "PHX", "SEA", "SFO",
                "from", "to", "flight_date", "target"]

y = flights["target"]
X = flights.drop(labels=useless_cols, axis=1)

# split df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=12654789)

In [6]:
# search_area = {
#     'nb_trees': (30, 200),
#     # 'max_depth': (7, 20), 
#     'min_samples_split': (0, 0.1),
#     'min_samples_leaf': (0, 0.1),
#     'ccp_alpha': (0, 0.1)
# }

search_area = {
    'max_depth': np.arange(7, 15),
    # 'learning_rate': [0.01, 0.1],
    # 'loss': ['squared_error', 'absolute_error']
}

In [7]:
regressor = GradientBoostingRegressor()

grid_search = GridSearchCV(
    regressor,
    param_grid=search_area,
    scoring="neg_root_mean_squared_error",
    cv=10,
    return_train_score=True,
    n_jobs=6,
    verbose=2
)

In [8]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


GridSearchCV(cv=10, estimator=GradientBoostingRegressor(), n_jobs=6,
             param_grid={'max_depth': array([ 7,  8,  9, 10, 11, 12, 13, 14])},
             return_train_score=True, scoring='neg_root_mean_squared_error',
             verbose=2)

In [9]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-0.567780388136899
{'max_depth': 8}


In [10]:
from sklearn.metrics import mean_squared_error

pred_train = grid_search.best_estimator_.predict(X_train)
pred_test = grid_search.best_estimator_.predict(X_test)

print(mean_squared_error(y_test, pred_test))
print(mean_squared_error(y_train, pred_train))

0.33548398886694747
0.1262584644895479


In [11]:
grid_search.cv_results_

{'mean_fit_time': array([ 9.07272029, 10.47867959, 13.31108549, 12.62085695, 12.62578714,
        14.10835447, 15.63107207, 16.39026968]),
 'std_fit_time': array([0.88832445, 0.58867942, 0.69452517, 1.05408017, 0.23937099,
        0.39694719, 0.3022611 , 2.03791818]),
 'mean_score_time': array([0.01555867, 0.00987313, 0.01196792, 0.01499093, 0.01416228,
        0.01396251, 0.01396284, 0.01595695]),
 'std_score_time': array([0.00195479, 0.00397527, 0.00558826, 0.00398604, 0.00413152,
        0.00430126, 0.00463498, 0.00655523]),
 'param_max_depth': masked_array(data=[7, 8, 9, 10, 11, 12, 13, 14],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 7},
  {'max_depth': 8},
  {'max_depth': 9},
  {'max_depth': 10},
  {'max_depth': 11},
  {'max_depth': 12},
  {'max_depth': 13},
  {'max_depth': 14}],
 'split0_test_score': array([-0.54825961, -0.53933413, -0.55976235, -0.57286682, -0.57531303